In [1]:
import pandas as pd
import urllib.request, json , time, os, difflib, itertools

import glob
import yfinance as yf
import os
from multiprocessing.dummy import Pool


In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        # print("True")
        return True
    except:
        conn.close()
        # print("False")
        return False

In [3]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [4]:
data_dax = yf.download("^GDAXI", start="1990-01-01")
data_dax

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GDAXI,^GDAXI,^GDAXI,^GDAXI,^GDAXI
Date,,,,,
1990-01-02,1788.890015,1788.890015,1788.890015,1788.890015,0
1990-01-03,1867.290039,1867.290039,1867.290039,1867.290039,0
1990-01-04,1830.920044,1830.920044,1830.920044,1830.920044,0
1990-01-05,1812.900024,1812.900024,1812.900024,1812.900024,0
1990-01-08,1841.469971,1841.469971,1841.469971,1841.469971,0
...,...,...,...,...,...
2025-09-23,23611.330078,23723.199219,23564.970703,23604.359375,48153500
2025-09-24,23666.810547,23736.599609,23499.980469,23601.500000,54163200


In [5]:
data_dax=data_dax.reset_index()
data_dax

Price,Date,Close,High,Low,Open,Volume
Ticker,,^GDAXI,^GDAXI,^GDAXI,^GDAXI,^GDAXI
0,1990-01-02,1788.890015,1788.890015,1788.890015,1788.890015,0
1,1990-01-03,1867.290039,1867.290039,1867.290039,1867.290039,0
2,1990-01-04,1830.920044,1830.920044,1830.920044,1830.920044,0
3,1990-01-05,1812.900024,1812.900024,1812.900024,1812.900024,0
4,1990-01-08,1841.469971,1841.469971,1841.469971,1841.469971,0
...,...,...,...,...,...,...
9039,2025-09-23,23611.330078,23723.199219,23564.970703,23604.359375,48153500
9040,2025-09-24,23666.810547,23736.599609,23499.980469,23601.500000,54163200
9041,2025-09-25,23534.830078,23629.369141,23383.839844,23620.400391,55308500


In [6]:
dax_close=data_dax["Close"].iloc[-1].values[0]

In [7]:
files=glob.glob('../data/*.xlsx')
files

['../data/~$Dax_MDax_Zusammensetzung_historisch.xlsx',
 '../data/Dax_Zusammensetzung_historisch_2.xlsx',
 '../data/Dax_MDax_Zusammensetzung_historisch_cleaned.xlsx',
 '../data/Dax_MDax_Zusammensetzung_historisch.xlsx']

In [8]:
base_data=pd.read_excel("../data/Dax_MDax_Zusammensetzung_historisch_cleaned.xlsx",skiprows=1)
base_data

,Status,Issuer,Date,Spalte1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379,Unnamed: 16380,Unnamed: 16381,Unnamed: 16382,Unnamed: 16383
0,Leaver,Covestro,2024-12-27,NaN,NaN,Showing data from 17-Sep-1993 to 10-Sep-2025 (...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
1,Joiner,Rheinmetall,2023-03-20,NaN,NaN,Status,Issuer,Code,Date,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
2,Joiner,Commerzbank,2023-02-27,NaN,NaN,Joiner,IONOS Group,IOSn.DE,2025-06-23 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
3,Leaver,Fresenius Med Cr,2023-03-20,NaN,NaN,2023-03-20 00:00:00,Leaver,Fresenius Med Cr,2023-03-20 00:00:00,Leaver,...,Leaver,Fresenius Med Cr,2023-03-20,Leaver,Fresenius Med Cr,2023-03-20,Leaver,Fresenius Med Cr,2023-03-20,Leaver
4,Leaver,Linde,2023-02-27,NaN,NaN,Leaver,Jenoptik,JENGn.DE,2025-06-23 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,NaN,NaN,NaT,NaN,NaN,Joiner,Varta,VARGk.F^K12 (expired),1993-09-17 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
613,NaN,NaN,NaT,NaN,NaN,Joiner,Villeroy & Boch,VIBG_p.F,1993-09-17 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
614,NaN,NaN,NaT,NaN,NaN,Joiner,Moksel,MKSG.F^H11 (expired),1993-09-17 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
615,NaN,NaN,NaT,NaN,NaN,Joiner,Wella,WADG_p.F^K07 (expired),1993-09-17 00:00:00,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN


In [9]:
dax_cases=base_data[base_data.columns[:3]].dropna()
dax_cases

,Status,Issuer,Date
0,Leaver,Covestro,2024-12-27
1,Joiner,Rheinmetall,2023-03-20
2,Joiner,Commerzbank,2023-02-27
3,Leaver,Fresenius Med Cr,2023-03-20
4,Leaver,Linde,2023-02-27
...,...,...,...
120,Joiner,Bayerische Hypo- und Wechselbank AG,1988-01-01
121,Joiner,Bayerische Vereinsbank AG,1988-01-01
122,Joiner,VIAG,1988-01-01
123,Joiner,Mannesmann,1988-01-01


In [10]:
dax_cases_joiner=dax_cases[dax_cases[dax_cases.columns[0]]=="Joiner"]
dax_cases_joiner

,Status,Issuer,Date
1,Joiner,Rheinmetall,2023-03-20
2,Joiner,Commerzbank,2023-02-27
5,Joiner,Porsche,2022-12-19
8,Joiner,Beiersdorf,2022-06-20
11,Joiner,Daimler Trk Hldg,2022-03-21
...,...,...,...
120,Joiner,Bayerische Hypo- und Wechselbank AG,1988-01-01
121,Joiner,Bayerische Vereinsbank AG,1988-01-01
122,Joiner,VIAG,1988-01-01
123,Joiner,Mannesmann,1988-01-01


In [11]:
dax_cases_leavers=dax_cases[dax_cases[dax_cases.columns[0]]=="Leaver"]
dax_cases_leavers

,Status,Issuer,Date
0,Leaver,Covestro,2024-12-27
3,Leaver,Fresenius Med Cr,2023-03-20
4,Leaver,Linde,2023-02-27
6,Leaver,Puma DE,2022-12-19
7,Leaver,Hellofresh,2022-09-19
9,Leaver,Delivery Hero,2022-06-20
10,Leaver,Beiersdorf,2022-03-21
14,Leaver,Deutsche Wohn,2021-10-29
25,Leaver,Beiersdorf,2021-03-22
28,Leaver,Wirecard,2020-08-24


In [12]:
len(dax_cases_leavers)

42

In [13]:
double_df=pd.merge(dax_cases_joiner,dax_cases_leavers,how="inner",on=["Issuer","Date"])
double_df

,Status_x,Issuer,Date,Status_y


In [14]:
dax_cases_leavers_cleaned=dax_cases_leavers.drop_duplicates().reset_index(drop=True)
dax_cases_leavers_cleaned

,Status,Issuer,Date
0,Leaver,Covestro,2024-12-27
1,Leaver,Fresenius Med Cr,2023-03-20
2,Leaver,Linde,2023-02-27
3,Leaver,Puma DE,2022-12-19
4,Leaver,Hellofresh,2022-09-19
5,Leaver,Delivery Hero,2022-06-20
6,Leaver,Beiersdorf,2022-03-21
7,Leaver,Deutsche Wohn,2021-10-29
8,Leaver,Beiersdorf,2021-03-22
9,Leaver,Wirecard,2020-08-24


In [15]:
for index, row in double_df.iterrows():
    print(row["Issuer"],row["Date"])
    dax_cases_leavers_cleaned=dax_cases_leavers_cleaned[~((dax_cases_leavers_cleaned["Issuer"]==row["Issuer"]) & (dax_cases_leavers_cleaned["Date"]==row["Date"]))]
dax_cases_leavers_cleaned=dax_cases_leavers_cleaned.reset_index(drop=True)
dax_cases_leavers_cleaned

,Status,Issuer,Date
0,Leaver,Covestro,2024-12-27
1,Leaver,Fresenius Med Cr,2023-03-20
2,Leaver,Linde,2023-02-27
3,Leaver,Puma DE,2022-12-19
4,Leaver,Hellofresh,2022-09-19
5,Leaver,Delivery Hero,2022-06-20
6,Leaver,Beiersdorf,2022-03-21
7,Leaver,Deutsche Wohn,2021-10-29
8,Leaver,Beiersdorf,2021-03-22
9,Leaver,Wirecard,2020-08-24


In [16]:
dax_cases_leavers=dax_cases_leavers_cleaned
dax_cases_leavers["Left"]=dax_cases_leavers["Date"]+pd.to_timedelta(0,unit='D')

dax_cases_leavers["SinceLeft_30"]=dax_cases_leavers["Date"]+pd.to_timedelta(30,unit='D')
dax_cases_leavers["SinceLeft_180"]=dax_cases_leavers["Date"]+pd.to_timedelta(180,unit='D')

dax_cases_leavers["SinceLeft_365"]=dax_cases_leavers["Date"]+pd.to_timedelta(365,unit='D')
dax_cases_leavers

,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365
0,Leaver,Covestro,2024-12-27,2024-12-27,2025-01-26,2025-06-25,2025-12-27
1,Leaver,Fresenius Med Cr,2023-03-20,2023-03-20,2023-04-19,2023-09-16,2024-03-19
2,Leaver,Linde,2023-02-27,2023-02-27,2023-03-29,2023-08-26,2024-02-27
3,Leaver,Puma DE,2022-12-19,2022-12-19,2023-01-18,2023-06-17,2023-12-19
4,Leaver,Hellofresh,2022-09-19,2022-09-19,2022-10-19,2023-03-18,2023-09-19
5,Leaver,Delivery Hero,2022-06-20,2022-06-20,2022-07-20,2022-12-17,2023-06-20
6,Leaver,Beiersdorf,2022-03-21,2022-03-21,2022-04-20,2022-09-17,2023-03-21
7,Leaver,Deutsche Wohn,2021-10-29,2021-10-29,2021-11-28,2022-04-27,2022-10-29
8,Leaver,Beiersdorf,2021-03-22,2021-03-22,2021-04-21,2021-09-18,2022-03-22
9,Leaver,Wirecard,2020-08-24,2020-08-24,2020-09-23,2021-02-20,2021-08-24


In [17]:
leavers_list=dax_cases_leavers.Issuer.tolist()
leavers_list

['Covestro',
 'Fresenius Med Cr',
 'Linde',
 'Puma DE',
 'Hellofresh',
 'Delivery Hero',
 'Beiersdorf',
 'Deutsche Wohn',
 'Beiersdorf',
 'Wirecard',
 'Lufthansa',
 'Thyssenkrupp',
 'Commerzbank',
 'Prosieben Media',
 'K&S',
 'Lanxess',
 'MetroOldData',
 'Man',
 'Salzgitter A',
 'Hannover Rueck',
 'Infineon Technol',
 'Postbank',
 'Continental',
 'Hypo Real Hldg',
 'TUI',
 'Altana',
 'Schering',
 'Hypo-Vereinsbank',
 'MLP S',
 'EpcosOldData',
 'Evonik Operation',
 'Dresdner Bk',
 'Karstadt-Quelle',
 'VIAG',
 'Mannesmann',
 'Hoechst',
 'Bayerische Vereinsbank AGOldData',
 'GEA Group',
 'Continental',
 'Kaufhof WH',
 'Babcock Borsig',
 'Diebold DE']

In [18]:
yahoo_xetra = {
    "Covestro": "1COV.DE",
    "Fresenius Med Cr": "FME.DE",
    "Linde": "LIN.DE",
    "Puma DE": "PUM.DE",
    "Hellofresh": "HFG.DE",
    "Delivery Hero": "DHER.DE",
    "Beiersdorf": "BEI.DE",
    "Siemens Energ": "ENR.DE",
    "Daimler Trk Hldg": "DTG.DE",
    "Deutsche Wohn": "DWNI.DE",
    "Vitesco Tech Grp": "VTSC.DE",
    "Wirecard": "WDI.HM",
    "Lufthansa": "LHA.DE",
    "Thyssenkrupp": "TKA.DE",
    "Bayerische Vereinsbank AGOldData":None,
    "EpcosOldData":None,
    "MetroOldData":None,
    "Linde DE": "LIN.DE",
    "Commerzbank": "CBK.DE",
    "Prosieben Media": "PSAN.DE",
    "Deutsche Boerse": "DB1.DE",
    "Uniper": "UN0.DE",
    "K&S": "SDF.DE",
    "Lanxess": "LXS.DE",
    "Osram Licht": "OSAGF",
    "Ceconomy": "CEC.DE",
    "Man": "MAN.DE",
    "Fresenius": "FRE.DE",
    "Salzgitter A": "SZG.DE",
    "Volkswagen": "VOW3.DE",
    "Bayer": "BAYN.DE",
    "Hannover Rueck": "HNR1.DE",
    "Infineon Technol": "IFX.DE",
    "Postbank": None,
    "Continental": "CON.DE",
    "Continent": "CON.DE",

    "Hypo Real Hldg": None,
    "TUI": "TUI1.DE",
    "Altana": "ALT.DE",
    "Bayer Pharma": "BAYN.DE",
    "UniCredt Bnk": None,
    "MLP S": "MLP.DE",
    "TDK Electronics": "TTDKY",
    "Evonik Operation": "EVK.DE",
    "Dresdner Bk": None,
    "Sap": "SAP.DE",
    "Arcandor": "ARO.DU",
    "VIAG": None,
    "Vodafone": None,
    "Mannesmann":None,  
    "Hoechst": None,
    "Mercedes Benz Gr": "MBG.DE",
    "Daimler-Benz AG": "MBG.DE",
    "GEA Group": "G1A.DE",
    "Kaufhof WH": None,
    "Babcock Borsig": None,
    "Diebold DE": None
}

In [19]:
dax_cases_leavers["Yahoo Symbol"]=dax_cases_leavers.Issuer.map(yahoo_xetra)
dax_cases_leavers

,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
0,Leaver,Covestro,2024-12-27,2024-12-27,2025-01-26,2025-06-25,2025-12-27,1COV.DE
1,Leaver,Fresenius Med Cr,2023-03-20,2023-03-20,2023-04-19,2023-09-16,2024-03-19,FME.DE
2,Leaver,Linde,2023-02-27,2023-02-27,2023-03-29,2023-08-26,2024-02-27,LIN.DE
3,Leaver,Puma DE,2022-12-19,2022-12-19,2023-01-18,2023-06-17,2023-12-19,PUM.DE
4,Leaver,Hellofresh,2022-09-19,2022-09-19,2022-10-19,2023-03-18,2023-09-19,HFG.DE
5,Leaver,Delivery Hero,2022-06-20,2022-06-20,2022-07-20,2022-12-17,2023-06-20,DHER.DE
6,Leaver,Beiersdorf,2022-03-21,2022-03-21,2022-04-20,2022-09-17,2023-03-21,BEI.DE
7,Leaver,Deutsche Wohn,2021-10-29,2021-10-29,2021-11-28,2022-04-27,2022-10-29,DWNI.DE
8,Leaver,Beiersdorf,2021-03-22,2021-03-22,2021-04-21,2021-09-18,2022-03-22,BEI.DE
9,Leaver,Wirecard,2020-08-24,2020-08-24,2020-09-23,2021-02-20,2021-08-24,WDI.HM


In [20]:
print(len(dax_cases_leavers[~dax_cases_leavers["Yahoo Symbol"].isna()]))
dax_cases_retrievable=dax_cases_leavers[~dax_cases_leavers["Yahoo Symbol"].isna()]
dax_cases_retrievable

27


,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
0,Leaver,Covestro,2024-12-27,2024-12-27,2025-01-26,2025-06-25,2025-12-27,1COV.DE
1,Leaver,Fresenius Med Cr,2023-03-20,2023-03-20,2023-04-19,2023-09-16,2024-03-19,FME.DE
2,Leaver,Linde,2023-02-27,2023-02-27,2023-03-29,2023-08-26,2024-02-27,LIN.DE
3,Leaver,Puma DE,2022-12-19,2022-12-19,2023-01-18,2023-06-17,2023-12-19,PUM.DE
4,Leaver,Hellofresh,2022-09-19,2022-09-19,2022-10-19,2023-03-18,2023-09-19,HFG.DE
5,Leaver,Delivery Hero,2022-06-20,2022-06-20,2022-07-20,2022-12-17,2023-06-20,DHER.DE
6,Leaver,Beiersdorf,2022-03-21,2022-03-21,2022-04-20,2022-09-17,2023-03-21,BEI.DE
7,Leaver,Deutsche Wohn,2021-10-29,2021-10-29,2021-11-28,2022-04-27,2022-10-29,DWNI.DE
8,Leaver,Beiersdorf,2021-03-22,2021-03-22,2021-04-21,2021-09-18,2022-03-22,BEI.DE
9,Leaver,Wirecard,2020-08-24,2020-08-24,2020-09-23,2021-02-20,2021-08-24,WDI.HM


In [49]:
dax_cases_non_retrievable=dax_cases_leavers[dax_cases_leavers["Yahoo Symbol"].isna()]
dax_cases_non_retrievable

,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
16,Leaver,MetroOldData,2012-09-24,2012-09-24,2012-10-24,2013-03-23,2013-09-24,None
21,Leaver,Postbank,2009-03-23,2009-03-23,2009-04-22,2009-09-19,2010-03-23,None
23,Leaver,Hypo Real Hldg,2008-12-22,2008-12-22,2009-01-21,2009-06-20,2009-12-22,None
26,Leaver,Schering,2006-09-18,2006-09-18,2006-10-18,2007-03-17,2007-09-18,NaN
27,Leaver,Hypo-Vereinsbank,2005-12-19,2005-12-19,2006-01-18,2006-06-17,2006-12-19,NaN
29,Leaver,EpcosOldData,2002-12-23,2002-12-23,2003-01-22,2003-06-21,2003-12-23,None
31,Leaver,Dresdner Bk,2001-07-20,2001-07-20,2001-08-19,2002-01-16,2002-07-20,None
32,Leaver,Karstadt-Quelle,2001-03-19,2001-03-19,2001-04-18,2001-09-15,2002-03-19,NaN
33,Leaver,VIAG,2000-06-19,2000-06-19,2000-07-19,2000-12-16,2001-06-19,None
34,Leaver,Mannesmann,2000-02-14,2000-02-14,2000-03-15,2000-08-12,2001-02-13,None


In [50]:
dictionary={}
for i in range(len(dax_cases_retrievable)):
    issuer=dax_cases_retrievable["Issuer"].iloc[i]
    print(issuer)
    ticker=dax_cases_retrievable["Yahoo Symbol"].iloc[i]
    date=dax_cases_retrievable["Date"].iloc[i]
    month_year=pd.to_datetime(date).strftime('%Y-%m')
    print(date)
    keyname=issuer+"_"+month_year
    print(keyname)
    if ticker is None:
        continue
    else:
        try:
            data = yf.download(ticker, start=date)
            print(data.head())
            data=data.reset_index()
        except:
            print("no data for ",issuer)
        try:
            close_now=data["Close"].iloc[-1].values[0]

            dictionary[keyname]={"Ticker":ticker,"Close now":close_now,"Issuer":issuer}
            for col in dax_cases_retrievable.columns[2:-1]:
                print(col)
                alt_date=dax_cases_retrievable[col].iloc[i]
                print(alt_date)
                value_alt=data[data["Date"]>=alt_date]["Close"].values[0][0]
                value_dax=data_dax[data_dax["Date"]>=alt_date]["Close"].values[0][0]
                print(value_alt)
                dictionary[keyname][col+"_date"]=alt_date
                dictionary[keyname][col]=value_alt
                dictionary[keyname][col+"_%"]=(close_now/value_alt-1)*100
                dictionary[keyname][col+"_dax"]=value_dax
                dictionary[keyname][col+"_dax%"]=(dax_close/value_dax-1)*100

        except:
            print(f"No trading data for {issuer} with ticker {ticker} since leaving date {date}")
            continue
dictionary

[*********************100%***********************]  1 of 1 completed

Covestro
2024-12-27 00:00:00
Covestro_2024-12
Price           Close       High        Low       Open  Volume
Ticker        1COV.DE    1COV.DE    1COV.DE    1COV.DE 1COV.DE
Date                                                          
2024-12-27  56.279999  58.060001  55.959999  57.900002  471114
2024-12-30  56.279999  56.279999  56.279999  56.279999       0
2025-01-02  55.860001  56.520000  55.720001  56.020000  210548
2025-01-03  55.720001  55.880001  55.500000  55.520000  135129
2025-01-06  56.040001  56.459999  55.419998  55.419998  162161
Date
2024-12-27 00:00:00
56.279998779296875
Left
2024-12-27 00:00:00
56.279998779296875
SinceLeft_30
2025-01-26 00:00:00
56.18000030517578
SinceLeft_180
2025-06-25 00:00:00
60.5
SinceLeft_365
2025-12-27 00:00:00
No trading data for Covestro with ticker 1COV.DE since leaving date 2024-12-27 00:00:00
Fresenius Med Cr
2023-03-20 00:00:00
Fresenius Med Cr_2023-03



[*********************100%***********************]  1 of 1 completed


Price           Close       High        Low       Open  Volume
Ticker         FME.DE     FME.DE     FME.DE     FME.DE  FME.DE
Date                                                          
2023-03-20  33.508217  34.272490  32.955733  33.922581  980523
2023-03-21  34.180408  34.336948  33.535842  33.609508  530860
2023-03-22  34.714478  35.009136  34.272490  34.281696  394482
2023-03-23  34.963093  35.018339  34.281693  34.511895  503982
2023-03-24  34.134365  34.824971  33.554255  34.824971  490514
Date
2023-03-20 00:00:00
33.508216857910156
Left
2023-03-20 00:00:00
33.508216857910156
SinceLeft_30
2023-04-19 00:00:00
37.56898498535156
SinceLeft_180
2023-09-16 00:00:00
39.814720153808594
SinceLeft_365
2024-03-19 00:00:00
33.966285705566406
Linde
2023-02-27 00:00:00
Linde_2023-02


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Price           Close       High        Low       Open  Volume
Ticker         G1A.DE     G1A.DE     G1A.DE     G1A.DE  G1A.DE
Date                                                          
2000-01-03  12.243127  12.474588  11.999483  12.121304  171431
2000-01-04  11.889842  12.230944  11.755837  12.060392  335095
2000-01-05  11.999485  12.176127  11.822842  11.822842  485247
2000-01-06  11.914207  12.060394  11.877661  12.060394  292804
2000-01-07  13.034971  13.034971  11.999484  12.029939  881688
Date
1996-11-19 00:00:00
12.24312686920166
Left
1996-11-19 00:00:00
12.24312686920166
SinceLeft_30
1996-12-19 00:00:00
12.24312686920166
SinceLeft_180
1997-05-18 00:00:00
12.24312686920166
SinceLeft_365
1997-11-19 00:00:00
12.24312686920166
Continental
1996-09-23 00:00:00
Continental_1996-09


[*********************100%***********************]  1 of 1 completed

Price          Close      High       Low      Open  Volume
Ticker        CON.DE    CON.DE    CON.DE    CON.DE  CON.DE
Date                                                      
1996-12-16  3.653553  3.677209  3.650935  3.664057  117953
1996-12-17  3.650937  3.650937  3.611536  3.631234   81642
1996-12-18  3.644385  3.677209  3.637809  3.677209   63602
1996-12-19  3.657512  3.657512  3.624682  3.650936  151605
1996-12-20  3.664057  3.723161  3.664057  3.703463  155421
Date
1996-09-23 00:00:00
3.653552532196045
Left
1996-09-23 00:00:00
3.653552532196045
SinceLeft_30
1996-10-23 00:00:00
3.653552532196045
SinceLeft_180
1997-03-22 00:00:00
4.74753475189209
SinceLeft_365
1997-09-23 00:00:00
6.073968410491943


{'Covestro_2024-12': {'Ticker': '1COV.DE',
  'Close now': np.float64(58.34000015258789),
  'Issuer': 'Covestro',
  'Date_date': Timestamp('2024-12-27 00:00:00'),
  'Date': np.float64(56.279998779296875),
  'Date_%': np.float64(3.660272597676051),
  'Date_dax': np.float64(19984.3203125),
  'Date_dax%': np.float64(18.930935307485196),
  'Left_date': Timestamp('2024-12-27 00:00:00'),
  'Left': np.float64(56.279998779296875),
  'Left_%': np.float64(3.660272597676051),
  'Left_dax': np.float64(19984.3203125),
  'Left_dax%': np.float64(18.930935307485196),
  'SinceLeft_30_date': Timestamp('2025-01-26 00:00:00'),
  'SinceLeft_30': np.float64(56.18000030517578),
  'SinceLeft_30_%': np.float64(3.844784328370876),
  'SinceLeft_30_dax': np.float64(21282.1796875),
  'SinceLeft_30_dax%': np.float64(11.678124193546612),
  'SinceLeft_180_date': Timestamp('2025-06-25 00:00:00'),
  'SinceLeft_180': np.float64(60.5),
  'SinceLeft_180_%': np.float64(-3.570247681672911),
  'SinceLeft_180_dax': np.float64(

In [51]:
dictionary

{'Covestro_2024-12': {'Ticker': '1COV.DE',
  'Close now': np.float64(58.34000015258789),
  'Issuer': 'Covestro',
  'Date_date': Timestamp('2024-12-27 00:00:00'),
  'Date': np.float64(56.279998779296875),
  'Date_%': np.float64(3.660272597676051),
  'Date_dax': np.float64(19984.3203125),
  'Date_dax%': np.float64(18.930935307485196),
  'Left_date': Timestamp('2024-12-27 00:00:00'),
  'Left': np.float64(56.279998779296875),
  'Left_%': np.float64(3.660272597676051),
  'Left_dax': np.float64(19984.3203125),
  'Left_dax%': np.float64(18.930935307485196),
  'SinceLeft_30_date': Timestamp('2025-01-26 00:00:00'),
  'SinceLeft_30': np.float64(56.18000030517578),
  'SinceLeft_30_%': np.float64(3.844784328370876),
  'SinceLeft_30_dax': np.float64(21282.1796875),
  'SinceLeft_30_dax%': np.float64(11.678124193546612),
  'SinceLeft_180_date': Timestamp('2025-06-25 00:00:00'),
  'SinceLeft_180': np.float64(60.5),
  'SinceLeft_180_%': np.float64(-3.570247681672911),
  'SinceLeft_180_dax': np.float64(

In [52]:
df_leavers_courses=pd.DataFrame.from_dict(dictionary,orient="index").reset_index().rename(columns={"index":"Key"})
df_leavers_courses

,Key,Ticker,Close now,Issuer,Date_date,Date,Date_%,Date_dax,Date_dax%,Left_date,...,SinceLeft_180_date,SinceLeft_180,SinceLeft_180_%,SinceLeft_180_dax,SinceLeft_180_dax%,SinceLeft_365_date,SinceLeft_365,SinceLeft_365_%,SinceLeft_365_dax,SinceLeft_365_dax%
0,Covestro_2024-12,1COV.DE,58.340000,Covestro,2024-12-27,56.279999,3.660273,19984.320312,18.930935,2024-12-27,...,2025-06-25,60.500000,-3.570248,23498.330078,1.145652,NaT,NaN,NaN,NaN,NaN
1,Fresenius Med Cr_2023-03,FME.DE,43.520000,Fresenius Med Cr,2023-03-20,33.508217,29.878593,14933.379883,59.157132,2023-03-20,...,2023-09-16,39.814720,9.306308,15727.120117,51.124547,2024-03-19,33.966286,28.127052,17987.490234,32.133715
2,Linde_2023-02,LIN.DE,404.799988,Linde,2023-02-27,314.395142,28.755166,15381.429688,54.521001,2023-02-27,...,2023-08-26,342.049011,18.345610,15792.610352,50.497850,2024-02-27,397.879730,1.739284,17556.490234,35.377509
3,Puma DE_2022-12,PUM.DE,20.670000,Puma DE,2022-12-19,48.570686,-57.443467,13942.870117,70.463749,2022-12-19,...,2023-06-17,47.296265,-56.296760,16201.200195,46.702335,2023-12-19,51.644680,-59.976516,16744.410156,41.943125
4,Hellofresh_2022-09,HFG.DE,7.448000,Hellofresh,2022-09-19,24.680000,-69.821719,12803.240234,85.636906,2022-09-19,...,2023-03-18,16.174999,-53.953630,14933.379883,59.157132,2023-09-19,33.040001,-77.457628,15664.480469,51.728869
5,Delivery Hero_2022-06,DHER.DE,24.760000,Delivery Hero,2022-06-20,36.480000,-32.127192,13265.599609,79.166715,2022-06-20,...,2022-12-17,42.650002,-41.946074,13942.870117,70.463749,2023-06-20,34.445000,-28.117287,16111.320312,47.520741
6,Beiersdorf_2022-03,BEI.DE,87.820000,Beiersdorf,2022-03-21,85.481972,2.735112,14326.969727,65.893692,2022-03-21,...,2022-09-17,99.625549,-11.849922,12803.240234,85.636906,2023-03-21,112.886116,-22.204782,15195.339844,56.413343
7,Deutsche Wohn_2021-10,DWNI.DE,21.049999,Deutsche Wohn,2021-10-29,43.961575,-52.117276,15688.769531,51.493965,2021-10-29,...,2022-04-27,27.464817,-23.356492,13793.940430,72.304203,2022-10-29,20.300800,3.690490,13253.740234,79.327033
8,Beiersdorf_2021-03,BEI.DE,87.820000,Beiersdorf,2021-03-22,85.127747,3.162604,14657.209961,62.155957,2021-03-22,...,2021-09-18,99.712791,-11.927047,15132.059570,57.067443,2022-03-22,89.192665,-1.538989,14473.200195,64.217580
9,Wirecard_2020-08,WDI.HM,0.018600,Wirecard,2020-08-24,1.216800,-98.471400,13066.540039,81.896194,2020-08-24,...,2021-02-20,0.410200,-95.465627,13950.040039,70.376135,2021-08-24,0.286700,-93.512382,15905.849609,49.426404


In [53]:
len(df_leavers_courses)

25

In [54]:
start_cols=["Issuer","Date_date"]

In [55]:
perc_cols=df_leavers_courses.columns[df_leavers_courses.columns.str.contains("%")].tolist()
perc_cols

['Date_%',
 'Date_dax%',
 'Left_%',
 'Left_dax%',
 'SinceLeft_30_%',
 'SinceLeft_30_dax%',
 'SinceLeft_180_%',
 'SinceLeft_180_dax%',
 'SinceLeft_365_%',
 'SinceLeft_365_dax%']

In [56]:
filtered_cols=start_cols+perc_cols
filtered_cols


['Issuer',
 'Date_date',
 'Date_%',
 'Date_dax%',
 'Left_%',
 'Left_dax%',
 'SinceLeft_30_%',
 'SinceLeft_30_dax%',
 'SinceLeft_180_%',
 'SinceLeft_180_dax%',
 'SinceLeft_365_%',
 'SinceLeft_365_dax%']

In [57]:
df_leavers_courses_red=df_leavers_courses[filtered_cols]
print(len(df_leavers_courses_red))
df_leavers_courses_red

25


,Issuer,Date_date,Date_%,Date_dax%,Left_%,Left_dax%,SinceLeft_30_%,SinceLeft_30_dax%,SinceLeft_180_%,SinceLeft_180_dax%,SinceLeft_365_%,SinceLeft_365_dax%
0,Covestro,2024-12-27,3.660273,18.930935,3.660273,18.930935,3.844784,11.678124,-3.570248,1.145652,NaN,NaN
1,Fresenius Med Cr,2023-03-20,29.878593,59.157132,29.878593,59.157132,15.840235,49.526516,9.306308,51.124547,28.127052,32.133715
2,Linde,2023-02-27,28.755166,54.521001,28.755166,54.521001,30.040105,55.051730,18.345610,50.497850,1.739284,35.377509
3,Puma DE,2022-12-19,-57.443467,70.463749,-57.443467,70.463749,-65.061697,56.552842,-56.296760,46.702335,-59.976516,41.943125
4,Hellofresh,2022-09-19,-69.821719,85.636906,-69.821719,85.636906,-65.486562,86.537744,-53.953630,59.157132,-77.457628,51.728869
5,Delivery Hero,2022-06-20,-32.127192,79.166715,-32.127192,79.166715,-40.495074,78.945746,-41.946074,70.463749,-28.117287,47.520741
6,Beiersdorf,2022-03-21,2.735112,65.893692,2.735112,65.893692,-5.896830,65.488713,-11.849922,85.636906,-22.204782,56.413343
7,Deutsche Wohn,2021-10-29,-52.117276,51.493965,-52.117276,51.493965,-48.211825,55.537964,-23.356492,72.304203,3.690490,79.327033
8,Beiersdorf,2021-03-22,3.162604,62.155957,3.162604,62.155957,-1.324058,56.406860,-11.927047,57.067443,-1.538989,64.217580
9,Wirecard,2020-08-24,-98.471400,81.896194,-98.471400,81.896194,-97.754437,87.990160,-95.465627,70.376135,-93.512382,49.426404


In [58]:
df_leavers_courses_red[df_leavers_courses_red["Left_%"]>df_leavers_courses_red["Left_dax%"]]

,Issuer,Date_date,Date_%,Date_dax%,Left_%,Left_dax%,SinceLeft_30_%,SinceLeft_30_dax%,SinceLeft_180_%,SinceLeft_180_dax%,SinceLeft_365_%,SinceLeft_365_dax%
12,Commerzbank,2018-09-24,263.984084,92.436927,263.984084,92.436927,329.622769,112.368880,404.285157,109.467448,557.127943,93.119758
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,1308.870778,319.280419,1248.014404,307.432568,1167.748544,296.952636,1106.572342,278.706419
18,Infineon Technol,2009-03-23,5557.043017,469.095612,5557.043017,469.095612,2440.872973,417.313164,1015.348574,319.280419,757.135047,294.988740
24,Continental,1996-09-23,1449.177081,805.713011,1449.177081,805.713011,1449.177081,787.215974,1092.197665,619.667973,831.845453,480.862046


In [59]:
overperfomer=df_leavers_courses_red[df_leavers_courses_red["Left_%"]>df_leavers_courses_red["Left_dax%"]]
overperfomer_red=overperfomer[["Issuer","Date_date","Left_%","Left_dax%"]]
overperfomer_red

,Issuer,Date_date,Left_%,Left_dax%
12,Commerzbank,2018-09-24,263.984084,92.436927
17,Hannover Rueck,2009-09-21,1308.870778,319.280419
18,Infineon Technol,2009-03-23,5557.043017,469.095612
24,Continental,1996-09-23,1449.177081,805.713011


In [60]:
overperfomer_red["Days_since_left"]=(pd.to_datetime("2025-09-25")-pd.to_datetime(overperfomer_red["Date_date"])).dt.days
overperfomer_red

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8116/4145584198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overperfomer_red["Days_since_left"]=(pd.to_datetime("2025-09-25")-pd.to_datetime(overperfomer_red["Date_date"])).dt.days


,Issuer,Date_date,Left_%,Left_dax%,Days_since_left
12,Commerzbank,2018-09-24,263.984084,92.436927,2558
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,5848
18,Infineon Technol,2009-03-23,5557.043017,469.095612,6030
24,Continental,1996-09-23,1449.177081,805.713011,10594


In [61]:
overperfomer_red["years_since_left"]=overperfomer_red["Days_since_left"]/365
overperfomer_red

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8116/1703169292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overperfomer_red["years_since_left"]=overperfomer_red["Days_since_left"]/365


,Issuer,Date_date,Left_%,Left_dax%,Days_since_left,years_since_left
12,Commerzbank,2018-09-24,263.984084,92.436927,2558,7.008219
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,5848,16.021918
18,Infineon Technol,2009-03-23,5557.043017,469.095612,6030,16.520548
24,Continental,1996-09-23,1449.177081,805.713011,10594,29.024658


In [62]:
overperfomer_red["yearly_interest"]=100*((overperfomer_red["Left_%"]+100)/100)**(1/overperfomer_red["years_since_left"])-100
overperfomer_red["yearly_interest_dax"]=100*((overperfomer_red["Left_dax%"]+100)/100)**(1/overperfomer_red["years_since_left"])-100

overperfomer_red

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8116/4272790986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overperfomer_red["yearly_interest"]=100*((overperfomer_red["Left_%"]+100)/100)**(1/overperfomer_red["years_since_left"])-100
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8116/4272790986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overperfomer_red["yearly_interest_dax"]=100*((overperfomer_red["Left_dax%"]+100)/100)**(1/overperfomer_red["years_since_left"])-100


,Issuer,Date_date,Left_%,Left_dax%,Days_since_left,years_since_left,yearly_interest,yearly_interest_dax
12,Commerzbank,2018-09-24,263.984084,92.436927,2558,7.008219,20.243227,9.790560
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,5848,16.021918,17.952247,9.358693
18,Infineon Technol,2009-03-23,5557.043017,469.095612,6030,16.520548,27.668993,11.099442
24,Continental,1996-09-23,1449.177081,805.713011,10594,29.024658,9.901370,7.887627


In [63]:
overperfomer_red.to_csv("overperformers_dax_leavers.csv")

In [64]:
len(df_leavers_courses_red[df_leavers_courses_red["Left_%"]>df_leavers_courses_red["Left_dax%"]])

4

In [65]:
df_leavers_courses_red[df_leavers_courses_red["SinceLeft_30_%"]>df_leavers_courses_red["SinceLeft_30_dax%"]]

,Issuer,Date_date,Date_%,Date_dax%,Left_%,Left_dax%,SinceLeft_30_%,SinceLeft_30_dax%,SinceLeft_180_%,SinceLeft_180_dax%,SinceLeft_365_%,SinceLeft_365_dax%
12,Commerzbank,2018-09-24,263.984084,92.436927,263.984084,92.436927,329.622769,112.368880,404.285157,109.467448,557.127943,93.119758
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,1308.870778,319.280419,1248.014404,307.432568,1167.748544,296.952636,1106.572342,278.706419
18,Infineon Technol,2009-03-23,5557.043017,469.095612,5557.043017,469.095612,2440.872973,417.313164,1015.348574,319.280419,757.135047,294.988740
24,Continental,1996-09-23,1449.177081,805.713011,1449.177081,805.713011,1449.177081,787.215974,1092.197665,619.667973,831.845453,480.862046


In [66]:
len(df_leavers_courses_red[df_leavers_courses_red["SinceLeft_30_%"]>df_leavers_courses_red["SinceLeft_30_dax%"]])

4

In [67]:
df_leavers_courses_red[df_leavers_courses_red["SinceLeft_180_%"]>df_leavers_courses_red["SinceLeft_180_dax%"]]

,Issuer,Date_date,Date_%,Date_dax%,Left_%,Left_dax%,SinceLeft_30_%,SinceLeft_30_dax%,SinceLeft_180_%,SinceLeft_180_dax%,SinceLeft_365_%,SinceLeft_365_dax%
11,Thyssenkrupp,2019-09-23,2.210349,92.569303,2.210349,92.569303,-5.605292,85.710153,226.024809,171.904017,161.732561,88.715291
12,Commerzbank,2018-09-24,263.984084,92.436927,263.984084,92.436927,329.622769,112.368880,404.285157,109.467448,557.127943,93.119758
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,1308.870778,319.280419,1248.014404,307.432568,1167.748544,296.952636,1106.572342,278.706419
18,Infineon Technol,2009-03-23,5557.043017,469.095612,5557.043017,469.095612,2440.872973,417.313164,1015.348574,319.280419,757.135047,294.988740
24,Continental,1996-09-23,1449.177081,805.713011,1449.177081,805.713011,1449.177081,787.215974,1092.197665,619.667973,831.845453,480.862046


In [68]:
len(df_leavers_courses_red[df_leavers_courses_red["SinceLeft_180_%"]>df_leavers_courses_red["SinceLeft_180_dax%"]])

5

In [69]:
df_leavers_courses_red[df_leavers_courses_red["SinceLeft_365_%"]>df_leavers_courses_red["SinceLeft_365_dax%"]]

,Issuer,Date_date,Date_%,Date_dax%,Left_%,Left_dax%,SinceLeft_30_%,SinceLeft_30_dax%,SinceLeft_180_%,SinceLeft_180_dax%,SinceLeft_365_%,SinceLeft_365_dax%
11,Thyssenkrupp,2019-09-23,2.210349,92.569303,2.210349,92.569303,-5.605292,85.710153,226.024809,171.904017,161.732561,88.715291
12,Commerzbank,2018-09-24,263.984084,92.436927,263.984084,92.436927,329.622769,112.368880,404.285157,109.467448,557.127943,93.119758
17,Hannover Rueck,2009-09-21,1308.870778,319.280419,1308.870778,319.280419,1248.014404,307.432568,1167.748544,296.952636,1106.572342,278.706419
18,Infineon Technol,2009-03-23,5557.043017,469.095612,5557.043017,469.095612,2440.872973,417.313164,1015.348574,319.280419,757.135047,294.988740
24,Continental,1996-09-23,1449.177081,805.713011,1449.177081,805.713011,1449.177081,787.215974,1092.197665,619.667973,831.845453,480.862046


In [70]:
len(df_leavers_courses_red[df_leavers_courses_red["SinceLeft_365_%"]>df_leavers_courses_red["SinceLeft_365_dax%"]])

5

In [71]:
dax_cases_leavers

,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
0,Leaver,Covestro,2024-12-27,2024-12-27,2025-01-26,2025-06-25,2025-12-27,1COV.DE
1,Leaver,Fresenius Med Cr,2023-03-20,2023-03-20,2023-04-19,2023-09-16,2024-03-19,FME.DE
2,Leaver,Linde,2023-02-27,2023-02-27,2023-03-29,2023-08-26,2024-02-27,LIN.DE
3,Leaver,Puma DE,2022-12-19,2022-12-19,2023-01-18,2023-06-17,2023-12-19,PUM.DE
4,Leaver,Hellofresh,2022-09-19,2022-09-19,2022-10-19,2023-03-18,2023-09-19,HFG.DE
5,Leaver,Delivery Hero,2022-06-20,2022-06-20,2022-07-20,2022-12-17,2023-06-20,DHER.DE
6,Leaver,Beiersdorf,2022-03-21,2022-03-21,2022-04-20,2022-09-17,2023-03-21,BEI.DE
7,Leaver,Deutsche Wohn,2021-10-29,2021-10-29,2021-11-28,2022-04-27,2022-10-29,DWNI.DE
8,Leaver,Beiersdorf,2021-03-22,2021-03-22,2021-04-21,2021-09-18,2022-03-22,BEI.DE
9,Leaver,Wirecard,2020-08-24,2020-08-24,2020-09-23,2021-02-20,2021-08-24,WDI.HM


In [72]:
merge_retrieved_and_not=pd.merge(df_leavers_courses,dax_cases_retrievable,left_on="Issuer",right_on="Issuer",how="outer",indicator=True)

In [78]:
print(len(dax_cases_non_retrievable))
dax_cases_non_retrievable

15


,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
16,Leaver,MetroOldData,2012-09-24,2012-09-24,2012-10-24,2013-03-23,2013-09-24,None
21,Leaver,Postbank,2009-03-23,2009-03-23,2009-04-22,2009-09-19,2010-03-23,None
23,Leaver,Hypo Real Hldg,2008-12-22,2008-12-22,2009-01-21,2009-06-20,2009-12-22,None
26,Leaver,Schering,2006-09-18,2006-09-18,2006-10-18,2007-03-17,2007-09-18,NaN
27,Leaver,Hypo-Vereinsbank,2005-12-19,2005-12-19,2006-01-18,2006-06-17,2006-12-19,NaN
29,Leaver,EpcosOldData,2002-12-23,2002-12-23,2003-01-22,2003-06-21,2003-12-23,None
31,Leaver,Dresdner Bk,2001-07-20,2001-07-20,2001-08-19,2002-01-16,2002-07-20,None
32,Leaver,Karstadt-Quelle,2001-03-19,2001-03-19,2001-04-18,2001-09-15,2002-03-19,NaN
33,Leaver,VIAG,2000-06-19,2000-06-19,2000-07-19,2000-12-16,2001-06-19,None
34,Leaver,Mannesmann,2000-02-14,2000-02-14,2000-03-15,2000-08-12,2001-02-13,None


In [79]:
dax_cases_non_retrievable[["Issuer","Date"]]

,Issuer,Date
16,MetroOldData,2012-09-24
21,Postbank,2009-03-23
23,Hypo Real Hldg,2008-12-22
26,Schering,2006-09-18
27,Hypo-Vereinsbank,2005-12-19
29,EpcosOldData,2002-12-23
31,Dresdner Bk,2001-07-20
32,Karstadt-Quelle,2001-03-19
33,VIAG,2000-06-19
34,Mannesmann,2000-02-14


In [89]:
non_found=merge_retrieved_and_not[merge_retrieved_and_not["_merge"]!="both"]
non_found

,Key,Ticker,Close now,Issuer,Date_date,Date_x,Date_%,Date_dax,Date_dax%,Left_date,...,SinceLeft_365_dax,SinceLeft_365_dax%,Status,Date_y,Left_y,SinceLeft_30_y,SinceLeft_180_y,SinceLeft_365_y,Yahoo Symbol,_merge
0,NaN,NaN,NaN,Altana,NaT,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,Leaver,2007-06-18,2007-06-18,2007-07-18,2007-12-15,2008-06-17,ALT.DE,right_only
24,NaN,NaN,NaN,Man,NaT,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,Leaver,2012-09-24,2012-09-24,2012-10-24,2013-03-23,2013-09-24,MAN.DE,right_only


In [90]:
non_found.rename(columns={"Date_y":"Date"},inplace=True)

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8116/3081664464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_found.rename(columns={"Date_y":"Date"},inplace=True)


In [92]:
non_found_summary=pd.concat([non_found[["Issuer","Date"]],dax_cases_non_retrievable[["Issuer","Date"]]],ignore_index=True)
non_found_summary.sort_values(by="Issuer",inplace=True)
non_found_summary

,Issuer,Date
0,Altana,2007-06-18
15,Babcock Borsig,1995-09-15
13,Bayerische Vereinsbank AGOldData,1998-06-22
16,Diebold DE,1990-09-03
8,Dresdner Bk,2001-07-20
7,EpcosOldData,2002-12-23
12,Hoechst,1999-09-20
4,Hypo Real Hldg,2008-12-22
6,Hypo-Vereinsbank,2005-12-19
9,Karstadt-Quelle,2001-03-19


In [93]:
dax_cases_retrievable

,Status,Issuer,Date,Left,SinceLeft_30,SinceLeft_180,SinceLeft_365,Yahoo Symbol
0,Leaver,Covestro,2024-12-27,2024-12-27,2025-01-26,2025-06-25,2025-12-27,1COV.DE
1,Leaver,Fresenius Med Cr,2023-03-20,2023-03-20,2023-04-19,2023-09-16,2024-03-19,FME.DE
2,Leaver,Linde,2023-02-27,2023-02-27,2023-03-29,2023-08-26,2024-02-27,LIN.DE
3,Leaver,Puma DE,2022-12-19,2022-12-19,2023-01-18,2023-06-17,2023-12-19,PUM.DE
4,Leaver,Hellofresh,2022-09-19,2022-09-19,2022-10-19,2023-03-18,2023-09-19,HFG.DE
5,Leaver,Delivery Hero,2022-06-20,2022-06-20,2022-07-20,2022-12-17,2023-06-20,DHER.DE
6,Leaver,Beiersdorf,2022-03-21,2022-03-21,2022-04-20,2022-09-17,2023-03-21,BEI.DE
7,Leaver,Deutsche Wohn,2021-10-29,2021-10-29,2021-11-28,2022-04-27,2022-10-29,DWNI.DE
8,Leaver,Beiersdorf,2021-03-22,2021-03-22,2021-04-21,2021-09-18,2022-03-22,BEI.DE
9,Leaver,Wirecard,2020-08-24,2020-08-24,2020-09-23,2021-02-20,2021-08-24,WDI.HM


In [94]:
non_found_summary.to_csv("leavers_not_retrieved.csv",sep=";",index=False)

In [181]:
still_to_retrieve=pd.read_csv("leavers_not_retrieved_hand.csv",sep="\t")  

In [182]:
still_to_retrieve

,Issuer,Ticker,Date of leaving,Close at leaving,Close now,Change (%),DAX Change (%),Status,Date,Yahoo Symbol,_merge
0,Altana,NaN,Aufgelöst,NaN,NaN,NaN,NaN,Leaver,2007-06-18,ALT.DE,right_only
1,Arcandor,NaN,Pennystock,NaN,NaN,NaN,NaN,Leaver,2001-03-19,NaN,right_only
2,Babcock Borsig,NaN,Aufgelöst,NaN,NaN,NaN,NaN,Leaver,1995-09-15,NaN,right_only
3,Daimler-Benz AG,MBG.DE,NaN,NaN,NaN,NaN,NaN,Leaver,1998-10-26,NaN,right_only
4,Diebold DE,NaN,Aufgekauft,NaN,NaN,NaN,NaN,Leaver,1990-09-03,DBN.DE,right_only
5,Dresdner Bk,NaN,Aufgekauft,NaN,NaN,NaN,NaN,Leaver,2001-07-20,NaN,right_only
6,Hoechst,NaN,Aufgelöst,NaN,NaN,NaN,NaN,Leaver,1999-09-20,NaN,right_only
7,Hypo Real Hldg,NaN,Verstaatlicht,NaN,NaN,NaN,NaN,Leaver,2008-12-22,NaN,right_only
8,Kaufhof WH,NaN,Aufgelöst,NaN,NaN,NaN,NaN,Leaver,1996-07-22,NaN,right_only
9,Man,NaN,Aufgelöst,NaN,NaN,NaN,NaN,Leaver,2012-09-24,MAN.DE,right_only


In [129]:
dax_absteiger_data=merge_retrieved_and_not[merge_retrieved_and_not["_merge"]=="both"]

In [130]:
#df_leavers_courses.drop_duplicates().to_csv("leavers_30_days.csv",index=False)

In [131]:
df_leavers_courses

,Key,Ticker,Close now,Issuer,Date_date,Date,Date_%,Date_dax,Date_dax%,Left_date,...,SinceLeft_180_date,SinceLeft_180,SinceLeft_180_%,SinceLeft_180_dax,SinceLeft_180_dax%,SinceLeft_365_date,SinceLeft_365,SinceLeft_365_%,SinceLeft_365_dax,SinceLeft_365_dax%
0,Covestro_2024-12,1COV.DE,58.380001,Covestro,2024-12-27,56.279999,3.731347,19984.320312,18.912972,2024-12-27,...,2025-06-25,60.500000,-3.504130,23498.330078,1.130375,NaT,NaN,NaN,NaN,NaN
1,Fresenius Med Cr_2023-03,FME.DE,43.549999,Fresenius Med Cr,2023-03-20,33.508217,29.968119,14933.379883,59.133092,2023-03-20,...,2023-09-16,39.814720,9.381653,15727.120117,51.101721,2024-03-19,33.966286,28.215371,17987.490234,32.113757
2,Linde_2023-02,LIN.DE,404.799988,Linde,2023-02-27,314.395142,28.755166,15381.429688,54.497662,2023-02-27,...,2023-08-26,342.049011,18.345610,15792.610352,50.475119,2024-02-27,397.879639,1.739307,17556.490234,35.357061
3,Puma DE_2022-12,PUM.DE,20.680000,Puma DE,2022-12-19,48.570686,-57.422878,13942.870117,70.438002,2022-12-19,...,2023-06-17,47.296265,-56.275616,16201.200195,46.680178,2023-12-19,51.644680,-59.957153,16744.410156,41.921686
4,Hellofresh_2022-09,HFG.DE,7.444000,Hellofresh,2022-09-19,24.680000,-69.837927,12803.240234,85.608868,2022-09-19,...,2023-03-18,16.174999,-53.978361,14933.379883,59.133092,2023-09-19,33.040001,-77.469735,15664.480469,51.705952
5,Delivery Hero_2022-06,DHER.DE,24.850000,Delivery Hero,2022-06-20,36.480000,-31.880481,13265.599609,79.139654,2022-06-20,...,2022-12-17,42.650002,-41.735054,13942.870117,70.438002,2023-06-20,34.445000,-27.856001,16111.320312,47.498459
6,Beiersdorf_2022-03,BEI.DE,88.160004,Beiersdorf,2022-03-21,85.481964,3.132871,14326.969727,65.868636,2022-03-21,...,2022-09-17,99.625549,-11.508640,12803.240234,85.608868,2023-03-21,112.886108,-21.903585,15195.339844,56.389719
7,Deutsche Wohn_2021-10,DWNI.DE,21.100000,Deutsche Wohn,2021-10-29,43.961575,-52.003538,15688.769531,51.471084,2021-10-29,...,2022-04-27,27.464815,-23.174431,13793.940430,72.278178,2022-10-29,20.300800,3.936791,13253.740234,79.299947
8,Beiersdorf_2021-03,BEI.DE,88.160004,Beiersdorf,2021-03-22,85.127762,3.561989,14657.209961,62.131465,2021-03-22,...,2021-09-18,99.712784,-11.586057,15132.059570,57.043720,2022-03-22,89.192673,-1.157796,14473.200195,64.192776
9,Wirecard_2020-08,WDI.HM,0.018600,Wirecard,2020-08-24,1.216800,-98.471400,13066.540039,81.868721,2020-08-24,...,2021-02-20,0.410200,-95.465627,13950.040039,70.350402,2021-08-24,0.286700,-93.512382,15905.849609,49.403834


In [132]:
#df_leavers_courses.drop_duplicates().to_csv("leavers_180_days.csv",index=False)

In [133]:
df_leavers_courses.drop_duplicates().to_csv("leavers_365_days.csv",index=False)

In [134]:
df_leavers_courses

,Key,Ticker,Close now,Issuer,Date_date,Date,Date_%,Date_dax,Date_dax%,Left_date,...,SinceLeft_180_date,SinceLeft_180,SinceLeft_180_%,SinceLeft_180_dax,SinceLeft_180_dax%,SinceLeft_365_date,SinceLeft_365,SinceLeft_365_%,SinceLeft_365_dax,SinceLeft_365_dax%
0,Covestro_2024-12,1COV.DE,58.380001,Covestro,2024-12-27,56.279999,3.731347,19984.320312,18.912972,2024-12-27,...,2025-06-25,60.500000,-3.504130,23498.330078,1.130375,NaT,NaN,NaN,NaN,NaN
1,Fresenius Med Cr_2023-03,FME.DE,43.549999,Fresenius Med Cr,2023-03-20,33.508217,29.968119,14933.379883,59.133092,2023-03-20,...,2023-09-16,39.814720,9.381653,15727.120117,51.101721,2024-03-19,33.966286,28.215371,17987.490234,32.113757
2,Linde_2023-02,LIN.DE,404.799988,Linde,2023-02-27,314.395142,28.755166,15381.429688,54.497662,2023-02-27,...,2023-08-26,342.049011,18.345610,15792.610352,50.475119,2024-02-27,397.879639,1.739307,17556.490234,35.357061
3,Puma DE_2022-12,PUM.DE,20.680000,Puma DE,2022-12-19,48.570686,-57.422878,13942.870117,70.438002,2022-12-19,...,2023-06-17,47.296265,-56.275616,16201.200195,46.680178,2023-12-19,51.644680,-59.957153,16744.410156,41.921686
4,Hellofresh_2022-09,HFG.DE,7.444000,Hellofresh,2022-09-19,24.680000,-69.837927,12803.240234,85.608868,2022-09-19,...,2023-03-18,16.174999,-53.978361,14933.379883,59.133092,2023-09-19,33.040001,-77.469735,15664.480469,51.705952
5,Delivery Hero_2022-06,DHER.DE,24.850000,Delivery Hero,2022-06-20,36.480000,-31.880481,13265.599609,79.139654,2022-06-20,...,2022-12-17,42.650002,-41.735054,13942.870117,70.438002,2023-06-20,34.445000,-27.856001,16111.320312,47.498459
6,Beiersdorf_2022-03,BEI.DE,88.160004,Beiersdorf,2022-03-21,85.481964,3.132871,14326.969727,65.868636,2022-03-21,...,2022-09-17,99.625549,-11.508640,12803.240234,85.608868,2023-03-21,112.886108,-21.903585,15195.339844,56.389719
7,Deutsche Wohn_2021-10,DWNI.DE,21.100000,Deutsche Wohn,2021-10-29,43.961575,-52.003538,15688.769531,51.471084,2021-10-29,...,2022-04-27,27.464815,-23.174431,13793.940430,72.278178,2022-10-29,20.300800,3.936791,13253.740234,79.299947
8,Beiersdorf_2021-03,BEI.DE,88.160004,Beiersdorf,2021-03-22,85.127762,3.561989,14657.209961,62.131465,2021-03-22,...,2021-09-18,99.712784,-11.586057,15132.059570,57.043720,2022-03-22,89.192673,-1.157796,14473.200195,64.192776
9,Wirecard_2020-08,WDI.HM,0.018600,Wirecard,2020-08-24,1.216800,-98.471400,13066.540039,81.868721,2020-08-24,...,2021-02-20,0.410200,-95.465627,13950.040039,70.350402,2021-08-24,0.286700,-93.512382,15905.849609,49.403834


In [135]:
df_leavers_courses.head(30)

,Key,Ticker,Close now,Issuer,Date_date,Date,Date_%,Date_dax,Date_dax%,Left_date,...,SinceLeft_180_date,SinceLeft_180,SinceLeft_180_%,SinceLeft_180_dax,SinceLeft_180_dax%,SinceLeft_365_date,SinceLeft_365,SinceLeft_365_%,SinceLeft_365_dax,SinceLeft_365_dax%
0,Covestro_2024-12,1COV.DE,58.380001,Covestro,2024-12-27,56.279999,3.731347,19984.320312,18.912972,2024-12-27,...,2025-06-25,60.500000,-3.504130,23498.330078,1.130375,NaT,NaN,NaN,NaN,NaN
1,Fresenius Med Cr_2023-03,FME.DE,43.549999,Fresenius Med Cr,2023-03-20,33.508217,29.968119,14933.379883,59.133092,2023-03-20,...,2023-09-16,39.814720,9.381653,15727.120117,51.101721,2024-03-19,33.966286,28.215371,17987.490234,32.113757
2,Linde_2023-02,LIN.DE,404.799988,Linde,2023-02-27,314.395142,28.755166,15381.429688,54.497662,2023-02-27,...,2023-08-26,342.049011,18.345610,15792.610352,50.475119,2024-02-27,397.879639,1.739307,17556.490234,35.357061
3,Puma DE_2022-12,PUM.DE,20.680000,Puma DE,2022-12-19,48.570686,-57.422878,13942.870117,70.438002,2022-12-19,...,2023-06-17,47.296265,-56.275616,16201.200195,46.680178,2023-12-19,51.644680,-59.957153,16744.410156,41.921686
4,Hellofresh_2022-09,HFG.DE,7.444000,Hellofresh,2022-09-19,24.680000,-69.837927,12803.240234,85.608868,2022-09-19,...,2023-03-18,16.174999,-53.978361,14933.379883,59.133092,2023-09-19,33.040001,-77.469735,15664.480469,51.705952
5,Delivery Hero_2022-06,DHER.DE,24.850000,Delivery Hero,2022-06-20,36.480000,-31.880481,13265.599609,79.139654,2022-06-20,...,2022-12-17,42.650002,-41.735054,13942.870117,70.438002,2023-06-20,34.445000,-27.856001,16111.320312,47.498459
6,Beiersdorf_2022-03,BEI.DE,88.160004,Beiersdorf,2022-03-21,85.481964,3.132871,14326.969727,65.868636,2022-03-21,...,2022-09-17,99.625549,-11.508640,12803.240234,85.608868,2023-03-21,112.886108,-21.903585,15195.339844,56.389719
7,Deutsche Wohn_2021-10,DWNI.DE,21.100000,Deutsche Wohn,2021-10-29,43.961575,-52.003538,15688.769531,51.471084,2021-10-29,...,2022-04-27,27.464815,-23.174431,13793.940430,72.278178,2022-10-29,20.300800,3.936791,13253.740234,79.299947
8,Beiersdorf_2021-03,BEI.DE,88.160004,Beiersdorf,2021-03-22,85.127762,3.561989,14657.209961,62.131465,2021-03-22,...,2021-09-18,99.712784,-11.586057,15132.059570,57.043720,2022-03-22,89.192673,-1.157796,14473.200195,64.192776
9,Wirecard_2020-08,WDI.HM,0.018600,Wirecard,2020-08-24,1.216800,-98.471400,13066.540039,81.868721,2020-08-24,...,2021-02-20,0.410200,-95.465627,13950.040039,70.350402,2021-08-24,0.286700,-93.512382,15905.849609,49.403834


In [136]:
df_leavers_head=df_leavers_courses.head(25)

In [95]:
data_thyssen = yf.download("TKA.DE", start="1990-01-01")
data_thyssen

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,TKA.DE,TKA.DE,TKA.DE,TKA.DE,TKA.DE
Date,,,,,
1999-03-26,10.460896,10.821615,10.200703,10.821615,617391
1999-03-29,10.644213,10.792049,10.171137,10.525943,776308
1999-03-30,10.608731,10.851183,10.389933,10.851183,604737
1999-03-31,10.898489,10.898489,10.537768,10.644210,1163251
1999-04-01,10.768394,10.928057,10.644212,10.732913,650035
...,...,...,...,...,...
2025-09-23,11.560000,11.945000,11.560000,11.890000,1888287
2025-09-24,11.635000,11.685000,11.410000,11.565000,1723689


In [97]:
data_thyssen=data_thyssen.reset_index()
data_thyssen

Price,Date,Close,High,Low,Open,Volume
Ticker,,TKA.DE,TKA.DE,TKA.DE,TKA.DE,TKA.DE
0,1999-03-26,10.460896,10.821615,10.200703,10.821615,617391
1,1999-03-29,10.644213,10.792049,10.171137,10.525943,776308
2,1999-03-30,10.608731,10.851183,10.389933,10.851183,604737
3,1999-03-31,10.898489,10.898489,10.537768,10.644210,1163251
4,1999-04-01,10.768394,10.928057,10.644212,10.732913,650035
...,...,...,...,...,...,...
6777,2025-09-23,11.560000,11.945000,11.560000,11.890000,1888287
6778,2025-09-24,11.635000,11.685000,11.410000,11.565000,1723689
6779,2025-09-25,11.400000,11.610000,11.310000,11.610000,1555297


In [101]:
data_thyssen[data_thyssen["Date"]>"2019-09-01"][["Date","Close"]].to_csv("thyssenkrupp_post_dax.csv",index=False)